기존 Dataset 코드에서

```python
np.save('D:/git/LipService/x_save', x_data) # x_save.npy
np.save('D:/git/LipService/y_data', y_data) # y_data.npy
```

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras import models, layers, activations
from keras.layers import Conv2D, TimeDistributed, BatchNormalization, MaxPooling2D, Flatten, Bidirectional, Dense, Dropout,Flatten
from keras.callbacks import EarlyStopping

In [2]:
x_data = np.load('lip/lip_x_data.npy') # x_save.npy
y_data = np.load('lip/lip_y_data.npy') # y_data.npy
y_data = keras.utils.to_categorical(y_data,11)
print(x_data.shape, y_data.shape)

(3000, 27, 64, 64, 3) (3000, 11)


In [3]:
# Train과 Test 분리
X_train, X_test, Y_train, Y_test = train_test_split(x_data, y_data, test_size = 0.2, shuffle = True, stratify=y_data)
#X_train = X_train / 255
print("X_train: {}, X_test: {}".format(X_train.shape, X_test.shape))
print("Y_train: {}, Y_test: {}".format(Y_train.shape, Y_test.shape))

X_train: (2400, 27, 64, 64, 3), X_test: (600, 27, 64, 64, 3)
Y_train: (2400, 11), Y_test: (600, 11)


In [4]:
import keras
from keras.applications.vgg16 import VGG16

class VGG_LSTM(keras.Model):
    def __init__(self, num_classes):
        super(VGG_LSTM, self).__init__()
        #self.BZ = input_shape[0]
        #self.frame = input_shape[1]

        self.vgg = TimeDistributed(VGG16(weights='imagenet', include_top=False, pooling=max))
        self.vgg.trainable = False

        self.bilstm = layers.Bidirectional(layers.LSTM(256), merge_mode='concat')
        self.dense = layers.Dense(num_classes, activation='softmax')
        self.max_pool = TimeDistributed(MaxPooling2D(pool_size=(2, 2)))
        self.dropout = layers.Dropout(0.5)
        self.flat= TimeDistributed(Flatten())

    def call(self, x):
        x = self.vgg(x)
        x = self.flat(x)
        x = self.dropout(x)
        x = self.bilstm(x)
        return self.dense(x)

In [5]:
def get_model_vgg():
    return VGG_LSTM(num_classes = 11)

In [6]:
model = get_model_vgg()
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
#history = model.fit(X_train, Y_train, batch_size=16, epochs=5, verbose = 1, validation_data = (X_test, Y_test))
#model.summary()

In [ ]:
es = EarlyStopping(monitor = 'val_loss', patience = 10)
history = model.fit(X_train, Y_train, batch_size=16, epochs=100, verbose = 1, validation_split=0.2, callbacks = [es])

Epoch 1/100
150/150 [==============================] - 2346s 16s/step - loss: 2.4286 - accuracy: 0.1085 - val_loss: 2.3278 - val_accuracy: 0.1000
Epoch 2/100
150/150 [==============================] - 2309s 15s/step - loss: 2.3242 - accuracy: 0.1022 - val_loss: 2.3257 - val_accuracy: 0.1100
Epoch 3/100
150/150 [==============================] - 2313s 15s/step - loss: 2.3062 - accuracy: 0.1242 - val_loss: 2.2958 - val_accuracy: 0.1100
Epoch 4/100
150/150 [==============================] - 2309s 15s/step - loss: 2.2859 - accuracy: 0.1367 - val_loss: 2.2867 - val_accuracy: 0.1100
Epoch 5/100
150/150 [==============================] - 2324s 16s/step - loss: 2.2684 - accuracy: 0.1451 - val_loss: 2.2699 - val_accuracy: 0.1217
Epoch 6/100
150/150 [==============================] - 2306s 15s/step - loss: 2.2163 - accuracy: 0.1735 - val_loss: 2.3073 - val_accuracy: 0.1217
Epoch 7/100
150/150 [==============================] - 2308s 15s/step - loss: 2.2071 - accuracy: 0.1815 - val_loss: 2.2727 -

In [ ]:
history.history

In [ ]:
import matplotlib.pyplot as plt
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()